In [0]:
!git clone https://github.com/hagaressmat/cnn.git

Cloning into 'cnn'...
remote: Enumerating objects: 80948, done.
remote: Total 80948 (delta 0), reused 0 (delta 0), pack-reused 80948
Receiving objects: 100% (80948/80948), 677.99 MiB | 37.11 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (81213/81213), done.


In [0]:
import numpy as np
import os
from imutils import paths
import cv2
import pandas as pd

def load_images(path):
  img_paths = list(paths.list_images(path))
  img_paths.sort();
  img_data = []
  labels = []
  for p in img_paths:
    d = cv2.imread(p)
    d = cv2.cvtColor(d, cv2.COLOR_BGR2RGB)
    d = cv2.resize(d, (64, 64))
    l = p.split(os.path.sep)[-2]
    img_data.append(d)
    labels.append(l)
  img_data = np.array(img_data)
  labels=pd.get_dummies(labels)
  labels = np.array(labels)
  return img_data, labels

In [0]:
trn_data_path = '/content/cnn/Training'
val_data_path = '/content/cnn/Test'
X_train, y_train = load_images(trn_data_path)
X_test, y_test = load_images(val_data_path)

print(X_train.shape)
print(y_train.shape)

(60486, 64, 64, 3)
(60486, 120)


In [0]:
#(2) import libraries
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

#(3) Building the Keras model
cnn = Sequential()

cnn.add(Conv2D(16, (5, 5), input_shape = (64, 64, 3), padding = "same", activation = "relu"))
cnn.add(MaxPooling2D())

cnn.add(Conv2D(32, (5,5), padding='same', activation='relu'))
cnn.add(MaxPooling2D())

cnn.add(Conv2D(64, (5,5), padding='same', activation='relu'))
cnn.add(MaxPooling2D())

cnn.add(Conv2D(128, (5,5), padding='same', activation='relu'))
cnn.add(MaxPooling2D())

cnn.add(Flatten())
cnn.add(Dropout(0.25))
cnn.add(Dense(256, activation = "relu"))
cnn.add(Dense(120, activation = "softmax"))


#(4) Compiling the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 16)        1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)        

In [0]:
#(5) Training the model
history_cnn = cnn.fit(X_train, y_train, epochs=8, batch_size=128, validation_split=0.1 , verbose=2, shuffle=True)

Train on 54437 samples, validate on 6049 samples
Epoch 1/8
 - 16s - loss: 1.2695 - accuracy: 0.7789 - val_loss: 37.4768 - val_accuracy: 0.0352
Epoch 2/8
 - 8s - loss: 0.0670 - accuracy: 0.9784 - val_loss: 39.1210 - val_accuracy: 0.0291
Epoch 3/8
 - 8s - loss: 0.0447 - accuracy: 0.9857 - val_loss: 33.5821 - val_accuracy: 0.0382
Epoch 4/8
 - 8s - loss: 0.0388 - accuracy: 0.9880 - val_loss: 40.7947 - val_accuracy: 0.0448
Epoch 5/8
 - 8s - loss: 0.0413 - accuracy: 0.9874 - val_loss: 46.7751 - val_accuracy: 0.0382
Epoch 6/8
 - 8s - loss: 0.0522 - accuracy: 0.9848 - val_loss: 40.6151 - val_accuracy: 0.0469
Epoch 7/8
 - 8s - loss: 0.0560 - accuracy: 0.9839 - val_loss: 47.7655 - val_accuracy: 0.0269
Epoch 8/8
 - 8s - loss: 0.0488 - accuracy: 0.9869 - val_loss: 48.8323 - val_accuracy: 0.0225


In [0]:
y_test = np.argmax(y_test , 1)

predictions = cnn.predict(X_test)
predictions = np.argmax(predictions , 1)

In [0]:
from sklearn.metrics import accuracy_score

print ('Test Accuracy Score :',accuracy_score(y_test, predictions))

Test Accuracy Score : 0.8764671646134445


In [0]:
from sklearn.metrics import confusion_matrix

print ('Test Confusion Matrix :\n',confusion_matrix(y_test, predictions))

Test Confusion Matrix :
 [[114   0   0 ...   0   0   0]
 [  0 148   0 ...   0   0   0]
 [  0   0 160 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
